# Lab on Stochastic Linear Bandits :

We provide the environment to run a standard linear bandit experiment. The objective of this lab session is to understand how to implement LinUCB, the algorithm seen in class and its variant LinTS. We shall see that in practice there are some shortcomings in the implementation to make it efficient so we will guide you to obtain a working version. 

Questions are inline in the notebook and some reserved space are allocated for answers, but feel free to add cells for remarks and run your own experiments to test hypotheses you may have. 


**Report instructions**: Please export this notebook as a .tex file. Then, edit it and properly compile it into a pdf and 
only submit what is below the section ``Linear Bandit Agents''. Please make sure your name and that of your co-author are either indicated in the section head below or in the tex title commands.
Finally, please check your document before submitting. Given the volume of reports, we cannot guarantee to follow up on corrupted or unreadable reports. 

Note: To export, you might need to follow the steps at https://nbconvert.readthedocs.io/en/latest/install.html#installing-tex.


In [ ]:

import numpy as np
from model import Environment, Agent
from display import plot_regret


from scipy.stats import bernoulli
from math import log

import random
import sys
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Action generators 
The provided function helps the environment to generate random action sets of size K in dimension d. All actions are renormalized. You can see an example below. 

In [ ]:
def randomActionsGenerator(K,d, mean=None):
    """
    K: int -- number of action vectors to be generated
    d : int -- dimension of the action space
    returns : an array of K vectors uniformly sampled on the unit sphere in R^d
    """
    if mean is None:
        mean = np.zeros(d)
    vecs = np.random.multivariate_normal(mean, np.eye(d), size=K)
    norms = np.linalg.norm(vecs,axis=1)
    return vecs / norms[:,np.newaxis]

In [ ]:
a = randomActionsGenerator(20,2)


In [ ]:
plt.scatter(a[:,0],a[:,1])

# Environment Class

The environment class allows to create 3 types of linear bandit problems: 
* 'fixed' : normally requires a fixed_actions input (otherwise randomly generated at start) which is kept all along the game;
* 'arbitrary': at each round, an 'arbitrary' set of actions is chosen and here we decided to simply create a pool of (d x K) vectors and let the environment choose K of them without replacement at each round;
* 'iid' : at each round, the environment samples K actions at random on the sphere.

For each of these types of game, the class is used to generate the action sets at each round and the reward for a chosen action (chosen by an Agent, see the "Play!" section for the details of the interaction).

In [ ]:
class LinearBandit(Environment):
    
    def __init__(self, theta, K, pb_type, model='gaussian', var=1., fixed_actions=None):
        """
        theta: d-dimensional vector (bounded) representing the hidden parameter
        K: number of actions per round (random action vectors generated each time)
        """
        self.model = model
        self.d = np.size(theta)
        self.theta = theta
        self.K = K
        self.var = var
        self.current_action_set = np.zeros(self.d)
        self.pb_type = pb_type
        if self.pb_type not in ['fixed', 'iid', 'arbitrary']:
            raise ValueError(pb_type, "this type of problem is unknown, either change it or define it :)")
            
        #Now, set up the game for the given type:
        if self.pb_type == 'fixed':
            if fixed_actions is not None:

                self.K, action_dim = fixed_actions.shape #safety reset of self.K in case different
                if action_dim != self.d: # safety check
                    raise ValueError(fixed_actions, "actions dimension and theta mismatch: fix your actions.")
                self.fixed_actions = fixed_actions
            else:
                #create a fixed action set:
                self.fixed_actions = randomActionsGenerator(self.K, self.d)
            
            self.current_action_set = self.fixed_actions
            
        elif self.pb_type == 'arbitrary':
            #generate d*K actions and then sample K from this set at each round
            # Note any other way to generate an arbitrary sequence can be used
            self.pool_of_actions = randomActionsGenerator(self.d * self.K, self.d)
            # select K actions without replacement in the pool:
            self.current_action_set = self.pool_of_actions[np.random.choice(self.d*self.K, self.K, replace=False)]
        elif self.pb_type == 'iid':
            # generate a random set
            self.current_action_set = randomActionsGenerator(self.K, self.d)       
            
        
    def get_action_set(self):
        if self.pb_type == 'fixed':   
            self.current_action_set = self.fixed_actions
            return self.current_action_set
        elif self.pb_type == 'iid':
            self.current_action_set = randomActionsGenerator(self.K, self.d)
            return self.current_action_set
        elif self.pb_type == 'arbitrary':
            self.current_action_set = self.pool_of_actions[np.random.choice(self.d*self.K, self.K, replace=False)]
            return self.current_action_set
        
        
    def get_reward(self, action):
    
        """ sample reward given action and the model of this bandit environment
        action: d-dimensional vector (action chosen by the learner)
        """
        mean = np.dot(action, self.theta)
        if self.model == 'gaussian':
            return np.random.normal(mean, scale=self.var)
        else:#add bernoulli model option
            raise NotImplementedError('only Gaussian rewards are implemented so far')
            
    def get_means(self):
        return np.dot(self.current_action_set, self.theta)
        
        


# Linear Bandit Agents

For your report, **please export only the notebook from this cell on**. 

**Your Name**: XX XX

**Your co-author's name**: XX XX / NA (leave NA if no co-author)



## LinUCB : Implementing optimism in $R^d$

As seen in class, the actions are now vectors in $R^d$, representing contextual features, and the environment is assumed to generate rewards according to some hidden linear function $f_\theta(a) = a^\top \theta$. The goal of the learner is thus to estimate $\theta$ while keeping a measure of the uncertainty in all the directions of the feature space. 

* **Baseline: Implementation of LinEpsilonGreedy** In the next cell, we implemented a LinUniform Agent that returns one of the action vectors of the action set, chosen uniformly at random. Please implement an LinEpsilonGreedy agent as seen in the previous Lab. Do you think these agents can have a sublinear regret ? 

*[Your answer]


* **Implementation of LinUCB**: you need to compute UCBs for each arm of the current action set received from the environment, but this time the exploration bonus depends on the history of taken actions and received rewards. 

* **Efficiency of the matrix inversion step**: One key step is to invert the covariance matrix in order to compute the elliptical norm of each available action. Remark however that at round $t+1$, the new covariance matrix is very similar to the previous one at rount $t$... Can you think of a way to optimize this step by simply updating the old one ? 
Hint : You can search for a way to compute the inverse of the sum of an invertible matrix A and the outer product, $uv^\top$, of vectors u and v.

*[Your answer]


* **Implement and additional exploration hyper-parameter**: It is common practice to modify LinUCB by multiplying the confidence bonus of each arm by some hyperparameter $ 0<\alpha <1 $. 
Please implement this little modification of LinUCB. 

You may wonder why one would do that and it is a legitimate question. We guide you to explore this question further below, bear with us :)

*[Your answer]



### Uniform random policy

In [ ]:
class LinUniform(Agent):
  def __init__(self):
    pass

  def get_action(self, arms):
    K, _ = arms.shape
    return arms[np.random.choice(K)]

  def receive_reward(self, chosen_arm, reward):
    pass

  def reset(self):
    pass

  #@staticmethod
  def name(self):
    return 'Unif'

  

### Lin-$\epsilon$-Greedy policy:

In [ ]:
class LinEpsilonGreedy(Agent):
  def __init__(self, d,lambda_reg, eps=0.1,):
    self.eps = eps # exploration probability
    self.d = d
    self.lambda_reg = lambda_reg
    self.reset()
    
  def reset(self):
    self.t = 0
    self.hat_theta = np.zeros(self.d)
    self.cov = self.lambda_reg * np.identity(self.d)
    self.invcov = np.identity(self.d)
    self.b_t = np.zeros(self.d) 

  def get_action(self, arms):
    K, _ = arms.shape
    ## TODO: implement here
    return arms[np.random.choice(K)]
    

  def receive_reward(self, chosen_arm, reward):
    """
    update the internal quantities required to estimate the parameter theta using least squares
    """
    
    # TODO: Implement updates here
    self.t += 1   
        

  #@staticmethod
  def name(self):
    return 'LinEGreedy('+str(self.eps)+')'


### Lin-UCB: The optimistic way

In [ ]:


class LinUCB(Agent):

    def __init__(self, d, delta, lambda_reg, alpha=1.):
        self.d = d
        self.delta = delta
        self.lambda_reg = lambda_reg
        self.cov = self.lambda_reg * np.identity(d)
        #TODO: instantiate alpha

        self.reset()

    def reset(self):
        # reset all local variables that should not be kept when the experiment is restarted
        self.t = 0
        self.hat_theta = np.zeros(self.d)
        self.cov = self.lambda_reg * np.identity(self.d)
        self.invcov = np.identity(self.d)
        self.b_t = np.zeros(self.d)  


    def get_action(self, arms):
        """
            This function implements LinUCB
            Input:
            -------
            arms : list of arms (d-dimensional vectors)

            Output:
            -------
            chosen_arm : vector (chosen arm array of features)
            """
        # compute the UCB of each of the arm in arms, here arms are vectors
        ucbs = np.array(K)

        for i in range(K):
            ucbs[i] = np.random.random()
            # use hat_theta and beta and the covariance matrix. 
        
        chosen_arm_index = np.argmax(self.UCBs)
        chosen_arm = arms[chosen_arm_index]
        return chosen_arm


#     return np.random.choice(K)
    

    def receive_reward(self, chosen_arm, reward):
        """
        update the internal quantities required to estimate the parameter theta using least squares
        """
        #TODO

        self.t += 1

        pass


    def name(self):
        return "LinUCB("+str(self.alpha)+')'

## LinTS : Taking the Bayesian way

Thompson Sampling is a popular bayesian alternative to the standard optimistic bandit algorithms (see Chapter 36 of Bandit Algorithms). The key idea is to rely on Bayesian *samples* to get a proxy for the hidden parameter $\theta$ of the problem instead of building high-probability confidence regions. 

* **Posterior derivation**: Let us place a Gaussian prior with mean $\mathbf{0}$ and covariance $\lambda I$ on $\theta$. Given actions $A_1,\ldots,A_t$ and rewards $Y_1,\ldots,Y_t$, Can you compute the expression of the posterior at the beginning of round $t+1$ ? 

*[Your answer]


* **Implementation of a LinTS (Linear Thompson Sampling) agent**.

In [ ]:
class LinTS(Agent):

  def __init__(self, d, delta, lambda_prior):
    self.d = d
    self.delta = delta
    self.lambda_prior = lambda_prior
    self.reset()

  def reset(self):
    # reset all local variables that should not be kept when the experiment is restarted
    self.t = 0
    self.hat_theta = np.zeros(self.d)
    self.cov = self.lambda_prior * np.identity(self.d)
    self.invcov = np.identity(self.d)
    self.b_t = np.zeros(self.d)  
    

  def get_action(self, arms):
    """
        This function implements LinUCB
        Input:
        -------
        arms : list of arms (d-dimensional vectors)

        Output:
        -------
        chosen_arm : vector (chosen arm array of features)
        """    
    
    ## TODO
    K, _ = arms.shape
    return arms[np.random.choice(K)]


    return chosen_arm

    

  def receive_reward(self, chosen_arm, reward):
    """
    update the internal quantities required to estimate the parameter theta using least squares
    """
    self.t += 1


  def name(self):
    return "LinTS"


# Play !
The function play runs one path of regret for one agent. The function experiment runs all agents several (Nmc) times and returns all the logged data. Feel free to check the inputs and outputs required when you decide on the implementation of your own agents.

In [ ]:
def play(environment, agent, Nmc, T):
    """
    Play one Nmc trajectories over a horizon T for the specified agent. 
    Return the agent's name (string) and the collected data in an nd-array.
    """
    
    data = np.zeros((Nmc, T))
    
    
    
    for n in range(Nmc):
        agent.reset()
        for t in range(T):
            action_set = environment.get_action_set()
            action = agent.get_action(action_set) 
            # Note that the main difference with the previous lab is that now get_action needs to receive the action_set
            reward = environment.get_reward(action)
            agent.receive_reward(action,reward)
            
            # compute instant regret
            means = environment.get_means()
            best_reward = np.max(means)
            data[n,t]= best_reward - reward # this can be negative due to the noise, but on average it's positive
            
    return agent.name(), data


def experiment(environment, agents, Nmc, T):
    """
    Play Nmc trajectories for all agents over a horizon T. Store all the data in a dictionary.
    """
    
    all_data = {}
    
    for agent in agents:
        agent_id, regrets = play(environment, agent,Nmc, T)
        
        all_data[agent_id] = regrets
        
    return all_data

### Default settings
These should be good but feel free to try others, please highlight your changes wherever necessary.

In [ ]:
d = 2  # dimension
K = 5  # number of arms

# parametor vector \theta, normalized :
# theta = randomActionsGenerator(1,d)
theta = np.array([0.45, 0.5])
theta /= np.linalg.norm(theta)

T = 1000  # Finite Horizon
N = 50  # Monte Carlo simulations

delta = 0.1

# save subsampled points for Figures
Nsub = 100
tsav = range(2, T, Nsub)

#choice of quantile display
q = 10




In [ ]:
# three environments

iid_env = LinearBandit(theta, K, pb_type='iid') 

fixed_actions = np.array(([1,0.1],[0.1,1],[0.3,0.4]))# or... #randomActionsGenerator(K,d)
fixed_env = LinearBandit(theta, K, pb_type='fixed', fixed_actions=fixed_actions) # check if gaps are small
print(fixed_env.get_means())

arbitrary_env = LinearBandit(theta, K, pb_type='arbitrary')


  
 

In [ ]:
# policies

linucb = LinUCB(d, delta, lambda_reg=1.)
uniform = LinUniform()
e_greedy = LinEpsilonGreedy(d, lambda_reg=1., eps=0.1)
greedy = LinEpsilonGreedy(d, lambda_reg=1., eps=0.)


# Example question: Which baseline is the strongest, LinEpsilonGreedy or LinUniform?
This is a running demo, please follow this "template" in the questions below: 1/ experiment setup (choose environment(s), choose policies and parameters, 2/ plot, 3/ conclude by making observations on your results and responding to the question. If something catches your attention and you want to investigate further, please concisely write up what you want to test and why, as well as addtional conclusions.

In [ ]:
## iid environment
e_greedy_vs_unif_iid = experiment(iid_env, [uniform, e_greedy], Nmc=N, T=T)

plot_regret(e_greedy_vs_unif_iid, q=10)

In [ ]:
## arbitrary environment
e_greedy_vs_unif_arb = experiment(arbitrary_env, [uniform, e_greedy], Nmc=N, T=T)

plot_regret(e_greedy_vs_unif_arb, q=10)

In [ ]:
## fixed-actions environment
e_greedy_vs_unif_fixed = experiment(fixed_env, [uniform, e_greedy], Nmc=N, T=T)

plot_regret(e_greedy_vs_unif_fixed, q=10)

**Observations**: We tested LinEspilonGreedy and Uniform on all three environment types and LinEpsilonGreedy is stronger (it has lower regret) in all cases. However, we remark that its regret seems linear in all cases as expected due to the non-diminishing amount of exploration, **perhaps it would be interesting to see what happens when we remove this forced exploration?** We note that the difference between Uniform and EGreedy is significantly smaller in the fixed action problem, which is likely due to the reasonably small gaps (~0.05) in this problem (see env setup in above cell). 

**We quickly run the experiment suggested above, to check how Greedy ($\epsilon=0$) performs:**

In [ ]:
greedy_iid = experiment(iid_env, [greedy, e_greedy], Nmc=N, T=T)
greedy_arb = experiment(arbitrary_env, [greedy, e_greedy], Nmc=N, T=T)
greedy_fixed = experiment(fixed_env, [greedy, e_greedy], Nmc=N, T=T)

plot_regret(greedy_iid, q=10)
plot_regret(greedy_arb, q=10)
plot_regret(greedy_fixed, q=10)

In the arbitrary and iid environments, Greedy seems to be learning! This is surprising and could be investigated further but it is beyond the present question's scope. In the fixed-actions environment, however, greedy has a linear regret. **We conclude that Greedy may actually be quite a strong baseline in iid and arbitrary environments**.

# Question: Is LinUCB better than the baseline LinEpsilonGreedy? 
Compare the two methods on all environments and conclude. In light of the above example, decide which values of epsilon you want to try in your comparisons and explain your choices following the suggested template.

**Conclusions**: TODO

# Question: What is the role of $\alpha$ for LinUCB ?

In the question above, we saw that the fixed-actions setting is the most interesting because it is the most difficult. For this question, we fix the environment to fixed-actions. 
Compare the behavior of various instantiations of LinUCB using different values of the scaling hyper-parameter $\alpha$ and conclude on its usefulness / risks.


**Conclusions**: TODO

# Question: Is Thompson Sampling better than LinUCB?
Compare these two policies in all environments and conclude. 


**Conclusions**: TODO